In [6]:
import pandas as pd 
import numpy as np 
import requests 
import json, io


In [27]:
df = pd.read_csv('DATASETS/lightning_2022.csv')
df['cx-cy'] = df.apply(lambda record: str(record.cx) + '-' + str(record.cy), axis =1 )

def weatherCall():
    
    co_ordinates = [(cxcy.split('-')[0], cxcy.split('-')[1]) for cxcy in df['cx-cy'].unique()]
    dfs = []
    for cx, cy in co_ordinates:

        json_obj = json.loads(requests.get(f'https://archive-api.open-meteo.com/v1/archive?latitude={cy}&longitude={cx}&start_date=2022-01-01&end_date=2022-12-25&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration&timezone=Asia%2FSingapore').content)
        data = pd.DataFrame(json_obj['daily']) 
        data['cx'] = cx
        data['cy'] = cy
        dfs.append(data)

    return pd.concat(dfs)

weather = weatherCall()


In [33]:
weather['PRIMARY_KEY'] = weather.apply(lambda x: f'{x.cx}-{x.cy}-{x.time}', axis = 1)
df['PRIMARY_KEY'] = df.apply(lambda x: f'{x.cx}-{x.cy}-{x.DATE}', axis = 1)

In [34]:
main_df = pd.merge(left = df, right = weather, how = 'inner', on = 'PRIMARY_KEY')

In [40]:
len(main_df.loc[main_df.Count != 0, :])/len(main_df)*100

5.794058827005558

In [39]:
main_df.to_csv('DATASETS/weather_lightning.csv', index=False)